# Tech Challenge 3

Fine tunning com dataset da amazon

O objetivo é utilizar o dataset fornecido com dados sobre produtos da amazon para treinar o modelo gpt-4o-mini da openai de formar que ele saiba responder perguntas sobre os produtos utilizados no treinamento. É uma forma de dar ao modelo conhecimento sobre informações "privadas" com as quais ele não foi originalmente treinado.

In [ ]:
!pip install openai tqdm

# Carregar o arquivo de dataset a partir do google drive

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#utilizando um dataset limitado em 100 mil linhas
dir_path = '/content/drive/My Drive/leandrodvd@gmail.com/FIAP Pós IA/tech challenge 3'
file_path = dir_path + '/trn_100k.json'

# Example: Reading a CSV file from Google Drive
import pandas as pd

# Replace 'your_file.csv' with the actual file name
df = pd.read_json(file_path, lines=True)

# Display the DataFrame
df.head()

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,0000032034,Adult Ballet Tutu Yellow,,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,0000913154,The Way Things Work: An Illustrated Encycloped...,,"[116, 117, 118, 119, 120, 121, 122]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
4,0001381245,Misty of Chincoteague,,[151],[1.0]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   uid         100000 non-null  object
 1   title       100000 non-null  object
 2   content     100000 non-null  object
 3   target_ind  100000 non-null  object
 4   target_rel  100000 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


In [ ]:
# remover linhas com content vazio

# Remove rows where 'title' or 'content' is NaN
df = df.dropna(subset=['title', 'content'])

# Remove rows where 'title' or 'content' is an empty string
df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')]


In [ ]:
df.head(100)

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
12,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...","[329, 330, 331, 332, 333, 334, 335, 336, 337, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
13,0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"[181, 182, 307, 380, 381]","[1.0, 1.0, 1.0, 1.0, 1.0]"
...,...,...,...,...,...
196,000599134X,Collins Bible Handbook,"Text: English, French (translation)","[427, 2507, 2508, 2509, 2510, 2511, 2512, 2513...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
197,0005476216,Songs Of Stamps Baxter - Volume 1,"Founded in 1947, Hal Leonard Corporation has b...","[250, 251, 252, 253, 254, 1857, 2302, 2470, 2471]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
198,0005235774,A Gift of Hope,The central theme here--that hope endures desp...,[2548],[1.0]
200,0005991897,Meditating As a Christian: Waiting upon God,"The theory, practice, and biblical origins of ...","[2563, 285864]","[1.0, 1.0]"


# separar um pequeno subset para testar chamadas de llm sem fine tuning

In [ ]:
test_df = df.head(500).copy()

In [ ]:
test_df.head(10)

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
12,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...","[329, 330, 331, 332, 333, 334, 335, 336, 337, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
13,0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"[181, 182, 307, 380, 381]","[1.0, 1.0, 1.0, 1.0, 1.0]"
15,0001516035,Worship with Don Moen [VHS],Worship with Don Moen [VHS],[422],[1.0]
16,0001837397,Autumn Story Brambly Hedge,"""!the most research-crammed fantasy ever set b...","[423, 424, 425, 427, 430, 438, 439, 440]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
17,0000791156,Spirit Led-Moving By Grace In The Holy Spirit'...,You can flow effortlessly and powerfully in th...,"[441, 442, 443, 444, 445, 446]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
18,0001714384,The Very Bad Bunny (Beginner Series),"By Marilyn Sadler, Illustrated by Roger Bollen","[483, 484, 485, 486, 487]","[1.0, 1.0, 1.0, 1.0, 1.0]"
22,0001983415,Nice for Mice,Jill Barklem was born in Epping in 1951. After...,[512],[1.0]


# Gerar algumas perguntas sobre os produtos

In [ ]:
import random

random.seed(42)

question_options = [
      'Please describe the ',
      'What can you tell me about the ',
      'Tell me about  ',
      'What is the product ',
      'Give me description of '
  ]

def get_question_for_product(title):
  question = random.choice(question_options) + '\"' + title + '\" from amazon'
  return question


In [ ]:
test_df['question'] = test_df['title'].apply(get_question_for_product)

In [ ]:
test_df.head(100)

,uid,title,content,target_ind,target_rel,question
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","Please describe the ""Girls Ballet Tutu Neon Pi..."
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]","Please describe the ""Mog's Kittens"" from amazon"
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","Tell me about ""Girls Ballet Tutu Neon Blue"" f..."
12,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...","[329, 330, 331, 332, 333, 334, 335, 336, 337, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","What can you tell me about the ""The Prophet"" f..."
13,0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"[181, 182, 307, 380, 381]","[1.0, 1.0, 1.0, 1.0, 1.0]","What can you tell me about the ""Rightly Dividi..."
...,...,...,...,...,...,...
196,000599134X,Collins Bible Handbook,"Text: English, French (translation)","[427, 2507, 2508, 2509, 2510, 2511, 2512, 2513...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","What is the product ""Collins Bible Handbook"" f..."
197,0005476216,Songs Of Stamps Baxter - Volume 1,"Founded in 1947, Hal Leonard Corporation has b...","[250, 251, 252, 253, 254, 1857, 2302, 2470, 2471]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","What is the product ""Songs Of Stamps Baxter - ..."
198,0005235774,A Gift of Hope,The central theme here--that hope endures desp...,[2548],[1.0],"What can you tell me about the ""A Gift of Hope..."
200,0005991897,Meditating As a Christian: Waiting upon God,"The theory, practice, and biblical origins of ...","[2563, 285864]","[1.0, 1.0]","Tell me about ""Meditating As a Christian: Wai..."


# Chamar o modelo gpt-4o-mini perguntando sobre os produtos

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key=openai_api_key)

def call_open_ai(question):
  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "user", "content": question}
    ]
    )
  return completion.choices[0].message.content

In [ ]:
from tqdm import tqdm
tqdm.pandas()

test_df['pre_test_response'] = test_df['question'].progress_apply(call_open_ai)

100%|██████████| 500/500 [19:05<00:00,  2.29s/it]


In [ ]:
test_df[['title','content','question', 'pre_test_response']].head(10)

,title,content,question,pre_test_response
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"Please describe the ""Girls Ballet Tutu Neon Pi...",Sure! While I can't browse the internet or acc...
3,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"Please describe the ""Mog's Kittens"" from amazon","""Mog's Kittens"" is a charming children's book ..."
7,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"Tell me about ""Girls Ballet Tutu Neon Blue"" f...",I don't have real-time access to current Amazo...
12,The Prophet,"In a distant, timeless place, a mysterious pro...","What can you tell me about the ""The Prophet"" f...","""The Prophet"" is a collection of 26 poetic ess..."
13,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"What can you tell me about the ""Rightly Dividi...","""Rightly Dividing the Word"" typically refers t..."
15,Worship with Don Moen [VHS],Worship with Don Moen [VHS],"What can you tell me about the ""Worship with D...","""Worship with Don Moen"" is a VHS recording fea..."
16,Autumn Story Brambly Hedge,"""!the most research-crammed fantasy ever set b...","Please describe the ""Autumn Story Brambly Hedg...","""Autumn Story"" is one of the charming books in..."
17,Spirit Led-Moving By Grace In The Holy Spirit'...,You can flow effortlessly and powerfully in th...,"Give me description of ""Spirit Led-Moving By G...","I'm sorry, but I cannot access external websit..."
18,The Very Bad Bunny (Beginner Series),"By Marilyn Sadler, Illustrated by Roger Bollen","Please describe the ""The Very Bad Bunny (Begin...","""The Very Bad Bunny (Beginner Series)"" is a ch..."
22,Nice for Mice,Jill Barklem was born in Epping in 1951. After...,"Give me description of ""Nice for Mice"" from am...","""Nice for Mice"" is a children's book often pra..."


# Preparar os dados para fine tunning do modelo

In [ ]:
def build_fine_tunning_prompt(row):
  return {"messages": [{"role": "user", "content": row['question']}, {"role": "assistant", "content": "This is what I know about '"+row['title']+ "' from amazon: "+row['content']}]}

In [ ]:
test_df['fine_tunning_prompt'] = test_df.apply(build_fine_tunning_prompt, axis=1)

In [ ]:
test_df.head(10)

,uid,title,content,target_ind,target_rel,question,pre_test_response,fine_tunning_prompt
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","Please describe the ""Girls Ballet Tutu Neon Pi...",Sure! While I can't browse the internet or acc...,"{'messages': [{'role': 'user', 'content': 'Ple..."
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]","Please describe the ""Mog's Kittens"" from amazon","""Mog's Kittens"" is a charming children's book ...","{'messages': [{'role': 'user', 'content': 'Ple..."
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","Tell me about ""Girls Ballet Tutu Neon Blue"" f...",I don't have real-time access to current Amazo...,"{'messages': [{'role': 'user', 'content': 'Tel..."
12,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...","[329, 330, 331, 332, 333, 334, 335, 336, 337, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","What can you tell me about the ""The Prophet"" f...","""The Prophet"" is a collection of 26 poetic ess...","{'messages': [{'role': 'user', 'content': 'Wha..."
13,0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"[181, 182, 307, 380, 381]","[1.0, 1.0, 1.0, 1.0, 1.0]","What can you tell me about the ""Rightly Dividi...","""Rightly Dividing the Word"" typically refers t...","{'messages': [{'role': 'user', 'content': 'Wha..."
15,0001516035,Worship with Don Moen [VHS],Worship with Don Moen [VHS],[422],[1.0],"What can you tell me about the ""Worship with D...","""Worship with Don Moen"" is a VHS recording fea...","{'messages': [{'role': 'user', 'content': 'Wha..."
16,0001837397,Autumn Story Brambly Hedge,"""!the most research-crammed fantasy ever set b...","[423, 424, 425, 427, 430, 438, 439, 440]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","Please describe the ""Autumn Story Brambly Hedg...","""Autumn Story"" is one of the charming books in...","{'messages': [{'role': 'user', 'content': 'Ple..."
17,0000791156,Spirit Led-Moving By Grace In The Holy Spirit'...,You can flow effortlessly and powerfully in th...,"[441, 442, 443, 444, 445, 446]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","Give me description of ""Spirit Led-Moving By G...","I'm sorry, but I cannot access external websit...","{'messages': [{'role': 'user', 'content': 'Giv..."
18,0001714384,The Very Bad Bunny (Beginner Series),"By Marilyn Sadler, Illustrated by Roger Bollen","[483, 484, 485, 486, 487]","[1.0, 1.0, 1.0, 1.0, 1.0]","Please describe the ""The Very Bad Bunny (Begin...","""The Very Bad Bunny (Beginner Series)"" is a ch...","{'messages': [{'role': 'user', 'content': 'Ple..."
22,0001983415,Nice for Mice,Jill Barklem was born in Epping in 1951. After...,[512],[1.0],"Give me description of ""Nice for Mice"" from am...","""Nice for Mice"" is a children's book often pra...","{'messages': [{'role': 'user', 'content': 'Giv..."


In [ ]:
import json
# save fine tunning prompt to jsonl file
prompts_file_path = dir_path + '/fine_tunning_prompt.jsonl'
# test_df[['fine_tunning_prompt']].to_json(prompts_file_path, orient='records', lines=True)
with open(prompts_file_path, 'w') as fine_tunning_prompt_file:
    for item in test_df['fine_tunning_prompt']:
        fine_tunning_prompt_file.write(f"{json.dumps(item)}\n")

In [ ]:
with open(prompts_file_path, 'r') as fine_tunning_prompt_file:
  print(fine_tunning_prompt_file.read())

{"messages": [{"role": "user", "content": "Please describe the \"Girls Ballet Tutu Neon Pink\" from amazon"}, {"role": "assistant", "content": "This is what I know about 'Girls Ballet Tutu Neon Pink' from amazon: High quality 3 layer ballet tutu. 12 inches in length"}]}
{"messages": [{"role": "user", "content": "Please describe the \"Mog's Kittens\" from amazon"}, {"role": "assistant", "content": "This is what I know about 'Mog's Kittens' from amazon: Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years."}]}
{"messages": [{"role": "user", "content": "Tell me about  \"Girls Ballet Tutu Neon Blue\" from amazon"}, {"role": "assistant",

# Executar o fine tunning do modelo gpt-4o-mini


In [ ]:
fine_tunning_file_create_response = client.files.create(
  file=open(prompts_file_path, "rb"),
  purpose="fine-tune"
)

In [ ]:
print(fine_tunning_file_create_response)
fine_tunning_file_id = fine_tunning_file_create_response.id
print('fine_tunning_file_id:'+fine_tunning_file_id)

FileObject(id='file-kVNwte1ocHRwSpwpcxZrjrfB', bytes=444655, created_at=1727550868, filename='fine_tunning_prompt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
fine_tunning_file_id:file-kVNwte1ocHRwSpwpcxZrjrfB


In [ ]:
fine_tunning_job_create_response = client.fine_tuning.jobs.create(
  training_file=fine_tunning_file_id,
  model="gpt-4o-mini-2024-07-18"
)

In [ ]:
print(fine_tunning_job_create_response)
fine_tunning_job_id = fine_tunning_job_create_response.id
print('fine_tunning_job_id:'+fine_tunning_job_id)

FineTuningJob(id='ftjob-j1PJxNbYMQDSOgLJtoAEzwsF', created_at=1727550899, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-c8uUqVLqiJsdnRvai2q3h9M0', result_files=[], seed=1898990731, status='validating_files', trained_tokens=None, training_file='file-kVNwte1ocHRwSpwpcxZrjrfB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
fine_tunning_job_id:ftjob-j1PJxNbYMQDSOgLJtoAEzwsF


In [ ]:
# Retrieve the state of a fine-tune
fine_tuning_job_retrieve_response = client.fine_tuning.jobs.retrieve(fine_tunning_job_id)
print(fine_tuning_job_retrieve_response)
fine_tuning_job_status = fine_tuning_job_retrieve_response.status
fine_tuned_model = fine_tuning_job_retrieve_response.fine_tuned_model
print(fine_tuning_job_status)
print('fine_tuned_model:'+str(fine_tuned_model))


FineTuningJob(id='ftjob-j1PJxNbYMQDSOgLJtoAEzwsF', created_at=1727550899, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::ACXd7sYK', finished_at=1727552571, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-c8uUqVLqiJsdnRvai2q3h9M0', result_files=['file-PLeLFJpmOiOyp3TRporklQxB'], seed=1898990731, status='succeeded', trained_tokens=276120, training_file='file-kVNwte1ocHRwSpwpcxZrjrfB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
succeeded
fine_tuned_model:ft:gpt-4o-mini-2024-07-18:personal::ACXd7sYK


# Test the fine tuned model

In [ ]:
def call_open_ai_fine_tuned(question):
  completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[{"role": "user", "content": question}]
  )
  return completion.choices[0].message.content

In [ ]:
test_df['fine_tune_test_response'] = test_df['question'].progress_apply(call_open_ai_fine_tuned)

100%|██████████| 500/500 [15:29<00:00,  1.86s/it]


In [1]:
test_df[['question','fine_tunning_prompt', 'fine_tune_test_response']].head(500)

NameError: name 'test_df' is not defined

In [ ]:
# make some new questions
def get_alternate_question_for_product(title):
  question = 'I\'m looking for some information about a product that\'s sold by amazon. The title of the product is: ' + '\"' + title + '\"'
  return question

In [ ]:
test_df['alternate_question'] = test_df['title'].apply(get_alternate_question_for_product)

In [ ]:
test_df.head()

In [ ]:
test_df['fine_tune_test_alternate_response'] = test_df['alternate_question'].progress_apply(call_open_ai_fine_tuned)

In [ ]:
test_df.head()

In [ ]:
test_df.to_csv(dir_path + '/test_df.csv', index=False)

In [ ]:
# try with some other questions
question = "What is the product 'A Day in the Life of Canada' from amazon"

print('question:'+question)
answer = call_open_ai_fine_tuned(question)
print('answer:'+answer)

question:What is the product 'A Day in the Life of Canada' from amazon
answer:This is what I know about 'A Day in the Life of Canada' from amazon: In a single memorable day, so the story goes, 30 of the world's leading photographers fanned out across Canada, snapping away to record a nation's character, largely in character-revealing candids. The project, which led to 125,000 exposures and this sampling of 185, all large, manages to suggest the country's surprising variety and unity conclusion that any guidebook could have cobbled together more easily, and without dubbing many of the snapshots "instantly forgettable." For all its squandered qualities, Canada succeeds in showing that not all country-in-a-day books are put-ons. 30,000 first printing; BOMC alternate.Forecast: The synchronicity of these efforts makes an intriguing editorial strategy, but a superficial one visually. Now that the America volume has hit the stands, Canada would do better as a co-pub.Mary Ann Tsiang, Berkeley 